# VGG16 pre trained

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import os
import numpy as np
import matplotlib.pyplot as plt
from contextlib import redirect_stdout
import pandas as pd

## Getting and saving the data

In [ ]:
# CONFIGURE
local_path = "/home/sciapps/Documents/Repos/tfm"
model_name = "VGG16_pretrained"

In [ ]:
# DATA SET DIRECTORIES
source_dir = "data/images/image_preprocessing/processed_images_train_test/"
train_dir = os.path.join(local_path, source_dir, "train")
test_dir = os.path.join(local_path, source_dir, "test")

In [ ]:
# OUTPUTS
save_dir = os.path.join(local_path, "models", "images", "outputs" model_name)
# Create outputs folder
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [ ]:
# LABELS
class_names = sorted(os.listdir(train_dir))
print(class_names)

## Image augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                            rotation_range=45,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            validation_split = 0.20)  

## Image decoding

In [ ]:
# CONFIGURATION  
img_height = 224 
img_width = 224
color_mode= "rgb"
class_mode="categorical"                                  
shuffle=True                                                               
seed = 1234 

In [ ]:
train_array = train_datagen.flow_from_directory(directory = train_dir,
                                            target_size=(img_width, img_height),
                                            color_mode = color_mode,
                                            shuffle = shuffle,
                                            class_mode = class_mode,
                                            subset = "training",
                                            seed=seed
                                            ) 
validation_array = train_datagen.flow_from_directory(train_dir,  # same directory as training data
                                                    target_size=(img_width, img_height),
                                                    color_mode = color_mode,
                                                    #batch_size=batch_size,
                                                    class_mode= class_mode,
                                                    subset='validation',
                                                    seed=seed) # set as validation data 

## Creating, training and evaluating the model

In [ ]:
# load pre-trained model with the weights
vgg16_model = tf.keras.applications.VGG16()
vgg16_model.summary()

In [ ]:
# Add the layers of vgg16 model to a new sequential model 
model = Sequential()
for layer in vgg16_model.layers[:-1]: # remove last layer
    model.add(layer)
# Rename model
model._name = model_name
model.name

In [ ]:
# Freeze the weights in the layers
for layer in model.layers:
    layer.trainable = False
# Add last layer for categories
model.add(Dense(len(class_names), activation = "softmax"))

In [ ]:
model.summary()

### Training the model

In [ ]:
# CONFIGURE
optimizer = "sgd" # Options: "sgd", "adam"
loss = "categorical_crossentropy"

# COMPILING THE MODEL
model.compile(optimizer=optimizer,
            loss = loss,
            metrics=['accuracy'])

In [ ]:
# CONFIGURE
epochs = 30  
steps_per_epoch = 4

# TRAINING THE MODEL
history = model.fit_generator(
    train_array,
    #batch_size = batch_size,
    steps_per_epoch= 4,
    epochs=epochs,
    verbose=1, # get a progress bar and ETA
    validation_data=validation_array,
    validation_steps=2 # batch_size
)

### Evaluating the model - Accuracy and loss

In [ ]:
history_dict = history.history
acc = history_dict["acc"]
val_acc = history_dict["val_acc"]
loss = history_dict["loss"]
val_loss = history_dict["val_loss"]
epochs_range = range(epochs)

In [ ]:
plt.figure(figsize=(8, 8))
plt.suptitle(model_name)
# Accuracy plots
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")
# Loss plots
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss") 
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.savefig(os.path.join(save_dir,"acc_loss_plot.png"))
plt.show()

### Saving model, model summary and outputs

In [ ]:
# EDIT FOR EACH MODEL
# Model description
model_description = f"""
{model_name}
# load pre-trained model with the weights
vgg16_model = tf.keras.applications.VGG16()
# Add the layers of vgg16 model to a new sequential model 
model = Sequential()
for layer in vgg16_model.layers[:-1]: # remove last layer
    model.add(layer)
# Freeze the weights in the layers
for layer in model.layers:
    layer.trainable = False
# Add last layer for categories
model.add(Dense(len(class_names), activation = "softmax"))
"""

In [ ]:
# Save model
model.save_weights(os.path.join(save_dir, "weights.h5")) 

In [ ]:
# Save model weights independently
model.save(os.path.join(save_dir, "model.h5"))

In [ ]:
# Save model description
with open(os.path.join(save_dir,"model_description.txt"), "w") as file:
    with redirect_stdout(file):
        print(model_description)

In [ ]:
# Save model summary
with open(os.path.join(save_dir,"model_summary.txt"), "w") as file:
    with redirect_stdout(file):
        model.summary()

In [ ]:
# Save model history to csv
history_df = pd.DataFrame(history.history) 
history_df.to_csv(os.path.join(save_dir, "model_history.csv"), sep=",", index=False)